# Model fitting

This notebook is used for fitting the experimental data to the mathematical model to determine the selection coefficient for both the plasmid experiment

## Import libraries, load data, and get host frequencies

In [ ]:
import module,plot,data
import lmfit
from lmfit import Minimizer, Parameters
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

Load experimental data from file

In [ ]:
dataset = pd.read_excel("expdata/plasmid-exp.xlsx", header=0)
dataset.head()

Get mean(std) of host frequencies data from dataset (ignore zeros)

In [ ]:
# get minimum day number for which data is available for all replicates.n
data_replicate1_days = [dataset.loc[(dataset['replicate']==2) 
                    * (dataset['replicate.n']==replicate ) 
                    ,['t']]
    for replicate in range(1,6+1)]
Dmax= min([max(data_replicate1_days[i]['t']) for i in range(0,6)])
# get host frequency data and store to array
hostfreq_data = [[[dataset.loc[(dataset['replicate']==2) 
                    * (dataset['replicate.n']==replicate ) 
                    * (dataset['t']==t ) 
                    ,['host_freq']]][0].to_numpy()
    for t in range(1,Dmax+1)]
    for replicate in range(1,6+1)]
hostfreq_data=np.array(hostfreq_data)[:,:,0,0]
hostfreq_data_nonzero=np.where(hostfreq_data!=0,hostfreq_data,np.nan)

hostfreq_data_nonzero=np.log10(hostfreq_data_nonzero)
hostfreq_mean=np.nanmean(hostfreq_data_nonzero,axis=0)
hostfreq_std=np.nanstd(hostfreq_data_nonzero,axis=0)
# get heterozygous frequency data and store to array
hetfreq_data = [[[dataset.loc[(dataset['replicate']==2) 
                    * (dataset['replicate.n']==replicate ) 
                    * (dataset['t']==t ) 
                    ,['hetero_freq']]][0].to_numpy()
    for t in range(1,Dmax+1)]
    for replicate in range(1,6+1)]
hetfreq_data=np.array(hetfreq_data)[:,:,0,0]
hetfreq_data_nonzero=np.where(hetfreq_data!=0,hetfreq_data,np.nan)
hetfreq_data_nonzero
hetfreq_mean=np.nanmean(hetfreq_data_nonzero,axis=0)
hetfreq_std=np.nanstd(hetfreq_data_nonzero,axis=0)


print('Mean(std) of host frequencies (over days 1..Dmax='+str(Dmax)+'):')
print(hostfreq_mean)
print('(',hostfreq_std,')')

print('Mean(std) of heterozygous frequencies (over days 1..Dmax='+str(Dmax)+'):')
print(hetfreq_mean)
print('(',hetfreq_std,')')



In [ ]:
Nc=pd.read_csv('expdata/Nc_plasmid.csv', index_col=0).squeeze()
print('Carrying capacities loaded:')

Nc

## Fitting

In [ ]:
# execution time ca. 3min 
# s_residual_array = []

def fcn2min(params,x,hostfreq_mean,hostfreq_std = None):
    s=params['s']
    print(s)

    sim=module.stochbottleSim(rwt=1-s,n=15,Nc=Nc[1:].values.tolist(),f=10**((-4)),
    D=30,b=0.01)
    ts_eod=sim[1]
    host_freq_eod=np.log10(np.sum(ts_eod[:,1:],axis=-1)/np.sum(ts_eod,axis=-1))
    
    dev=host_freq_eod-hostfreq_mean
    eps=hostfreq_std
    res=dev/eps
    
    s_residual_array.append([s.value,np.sum(np.square(res)),host_freq_eod])

    figax=plt.subplots()
    # lim=(1e-6,1e-0); plt.ylim(*lim);
    plt.errorbar(x=range(1,len(hostfreq_mean)+1),y=hostfreq_mean,yerr=hostfreq_std,
                 label='experiment', color='black', alpha=1.)
    plt.errorbar(x=range(1,len(host_freq_eod)+1),y=host_freq_eod)

    plt.legend()
    plt.show()
    # print('host_freq_eod (sim)',host_freq_eod)
    # print('hostfreq_mean (exp)',hostfreq_mean)
    print('dev',dev)
    print('eps',eps)
    print('res',res)
    print('chisquared',np.sum(np.square(res)),'\n\n')
    print('reduced chisquared',np.sum(np.square(res))/(30-1),'\n\n')
    print('regression standard error',np.sqrt(np.sum(np.square(res))/(30-1)),'\n\n')
     
    print(hostfreq_std)
    return res
    # if hostfreq_std.any(None):
    #     return dev
    # else:
    #     return dev / np.log10(hostfreq_std)

x=np.array(range(0,len(hostfreq_mean)))

sspace=np.concatenate((np.linspace(0.05,0.20,16),np.linspace(0.08,0.09,11)))
sspace=np.linspace(0.06,0.11,21)
print(sspace);print(len(sspace))
for s in sspace:
    print(s)
    params=Parameters()
    params.add('s',value=s)
    fcn2min(params,x,hostfreq_mean,hostfreq_std)

params=Parameters()
params.add('s',value=0.1,min=0.00,max=1.0)
minner=Minimizer(fcn2min,params,fcn_args=(x,hostfreq_mean,hostfreq_std)) #hostfreq_std
result=minner.minimize(method='least_squares') 
result


## Result

In [ ]:
result

In [ ]:
plt.figure(figsize=(2.95,2.95*2/3))
plt.rcParams.update({'font.size': 8})

plt.plot([s_residual_array[i][0] for i in range(0,len(s_residual_array))], # x-values
         [(s_residual_array[i][1]/29)**0.5 for i in range(0,len(s_residual_array))], # y-values
         linestyle='none', marker='x', color='black',markersize=2.5
        )

# plt.yscale('log')
ylim=plt.gca().get_ylim(); print('auto-set ylim:',ylim)
plt.xlim(0.045,0.175)
# plt.xlim(0.075,0.115)
ylim=(-4/20,4+4/20)
ylim=(-4/20,4+4/20)
plt.ylim(*ylim)

#exp data
plt.plot(np.ones(2)*0.118471807,ylim)
sem=0.041506276/np.sqrt(6)
plt.fill_betweenx(ylim, np.ones(2)*0.118471807-sem*1.96, np.ones(2)*0.118471807+sem*1.96, alpha=0.25,linewidth=0)


# ylim=(-0.05,3.05) # manually set ylim
plt.plot(np.ones(2)*result.params['s'].value, ylim, color='black'); plt.ylim(ylim)
plt.fill_betweenx(ylim, 
                  np.ones(2)*result.params['s'].value-result.params['s'].stderr*1.96, 
                  np.ones(2)*result.params['s'].value+result.params['s'].stderr*1.96, 
                  alpha=0.25,color='black',linewidth=0)

plt.xlabel('Strength of selection $s$')
plt.ylabel('Regression standard error')
plt.text(-0.25, 1.06, s='A', transform=plt.gca().transAxes, 
                     size=11,weight='bold')
plt.title('Polyploid replicon')
plt.tight_layout()
plt.savefig('../figures-plots/plot_SI-fit_A.pdf')

In [ ]:
s_residual_array1=np.array(s_residual_array)
plt.plot(s_residual_array1[:,1]-s_residual_array1[-1,1])
plt.ylim(-.5,10)